In [5]:
import datumaro as dm
import os
from os import path as osp
import cv2
from matplotlib import pyplot as plt
import glob
from ultralytics import YOLO
from ultralytics import settings
from pathlib import Path

from collections import defaultdict
import numpy as np
import pandas as pd

import yaml

# Update a setting
settings.update({'runs_dir': '/training/runs'})

DUP_LABELS_MAPPING = {
        'White Fish': 'Whitefish',
        'Bull Trout': 'Bull',
        'Lan prey': 'Lamprey',
        'Lampray': 'Lamprey'
        }

# DataLoader
* data_frames
* load()
* next_frame()
# SalmonCounter
* DataLoader field
* model
* track_history
* LINE_OF_INTEREST_RATIO
* load()
* count()
# Evaluator
* evaluate()

In [6]:
def track_img_frames(model_path, frames_folder):
    botsort='botsort.yaml'
    bytetrack='bytetrack.yaml'
    
    model = YOLO(model_path)
    frames = glob.glob(osp.join(frames_folder, '*.jpg'))
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter('output.mp4', fourcc, 25.0, (1920, 1080))
    for frame in sorted(frames):
        results = model.track(frame, persist=True, tracker=bytetrack)
        
        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        out.write(annotated_frame)

    out.release()

track_img_frames('train_57_no_filt/weights/best.pt', '/mnt/shiorissd4tb/masamim/export_kitwanga_all_yolo/test/09-02-2020_07-49-21_m_salmon_camera/obj_train_data/')
#track_img_frames('train_57_no_filt/weights/best.pt', '/home/masamim/salmon-computer-vision/utils/export_bear_creek_all_yolo/test/09-04-2021_06-42-05_m_network_camera/obj_train_data/')

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'

image 1/1 /mnt/shiorissd4tb/masamim/export_kitwanga_all_yolo/test/09-02-2020_07-49-21_m_salmon_camera/obj_train_data/frame_000000.jpg: 384x640 (no detections), 5.7ms
Speed: 1.6ms preprocess, 5.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /mnt/shiorissd4tb/masamim/export_kitwanga_all_yolo/test/09-02-2020_07-49-21_m_salmon_camera/obj_train_data/frame_000001.jpg: 384x640 (no detections), 5.3ms
Speed: 1.3ms preprocess, 5.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /mnt/shiorissd4tb/masamim/export_kitwanga_all_yolo/test/09-02-2020_07-49-21_m_salmon_camera/obj_train_data/frame_000002.jpg: 384x640 (no detections), 5.3ms
Speed: 1.5ms preprocess, 5.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /mnt/shiorissd4tb/masamim/export_k

In [108]:
from abc import ABC, abstractmethod
from ultralytics.engine.results import Boxes

class DataLoader(ABC):
    @abstractmethod
    def next_clip(self):
        pass
    
    @abstractmethod
    def items(self):
        pass

    @abstractmethod
    def classes(self) -> dict:
        ### Expects return format {0: class1, 1: class2, ...}
        pass

class Item():
    def __init__(self, frame, boxes: list[Boxes]=None, attributes: list[dict]=None):
        self.frame = frame # Can be image or path to file
        self.boxes = boxes
        self.attrs = attributes # Extra attributes if needed

class ImageDirLoader(DataLoader):
    IMG_PATTERN = '*.[jpJP][npNP]*[gG$]'
    def __init__(self, root_folder, img_folder_name, custom_classes: dict):
        path = Path(root_folder).resolve()
        if not path.is_dir():
            raise ValueError(f"'{path}' is not a directory.")
        self.root_dir = path
        self.img_folder_name = img_folder_name
        self.custom_classes = custom_classes
        self.clip_gen = self.root_dir.iterdir()
        self.cur_clip = False

    def next_clip(self):
        self.cur_clip = next(self.clip_gen)
        return self.cur_clip
        
    def items(self):
        if not self.cur_clip:
            raise ValueError('No current clip')
        frec = lambda p, g: sorted(p.glob(g))
        for f in frec(self.cur_clip / self.img_folder_name, self.IMG_PATTERN):
            item = Item(f)
            yield item

    def classes(self) -> dict:
        return self.custom_classes

#with open('./export_combined_bear_kitwanga_all_yolo/combined_bear_kitwanga_all.yaml', 'r') as file:
#    data = yaml.safe_load(file)
#
#d = ImageDirLoader('/mnt/shiorissd4tb/masamim/export_kitwanga_all_yolo/test/', 'obj_train_data', data['names'])

In [ ]:
import json

class DatumaroLoader(DataLoader):
    def __init__(self, root_folder, custom_classes: dict):
        path = Path(root_folder).resolve()
        if not path.is_dir():
            raise ValueError(f"'{path}' is not a directory.")
        self.root_dir = path
        self.custom_classes = custom_classes
        self.clip_gen = self.root_dir.iterdir()
        self.cur_clip = False
        
    def next_clip(self):
        self.cur_clip = next(self.clip_gen)
        return self.cur_clip
        
    def items(self):
        if not self.cur_clip:
            raise ValueError('No current clip')
        frec = lambda p, g: sorted(p.glob(g))
        for f in frec(self.cur_clip / self.img_folder_name, self.IMG_PATTERN):
            item = Item(f)
            yield item

    def classes(self) -> dict:
        return self.custom_classes

In [102]:
class SalmonCounter:
    LEFT_PRE = 'l_'
    RIGHT_PRE = 'r_'
    FILENAME = 'filename'
    def __init__(self, model_path: str, dataloader: DataLoader, tracking_thresh = 10):
        self.model = YOLO(model_path)
        self.dataloader = dataloader
        self.track_history = defaultdict(lambda: [])
        classes = dataloader.classes()
        cols = [self.FILENAME]
        for i in range(len(classes)):
            cols.append(self.LEFT_PRE + classes[i])
            cols.append(self.RIGHT_PRE + classes[i])
        self.salm_count = pd.DataFrame(columns=cols).set_index(self.FILENAME)
        self.prev_track_ids = {}
        self.tracking_thresh = tracking_thresh
        

    def count(self, use_gt=False):
        cur_clip = self.dataloader.next_clip()
        self.salm_count.loc[cur_clip.name] = 0
        #fourcc = cv2.VideoWriter_fourcc(*'MP4V')
        #out_vid = cv2.VideoWriter('output.mp4', fourcc, 25.0, (1920, 1080))
        for item in self.dataloader.items():
            # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = self.model.track(item.frame, persist=True)
    
            # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            id_items = results[0].boxes.id
            track_ids = []
            if id_items:
                track_ids = id_items.int().cpu().tolist()
    
            # Visualize the results on the frame
            #annotated_frame = results[0].plot()
    
            # When any tracking ID is lost
            # Set difference prev track IDs - current track IDs
            not_tracking = set(self.prev_track_ids.keys()).difference(track_ids)
            for track_id in not_tracking:
                if self.prev_track_ids[track_id] > 0:
                    # Each tracking ID has a counter
                    # Decrement counter for that tracking ID
                    self.prev_track_ids[track_id] -= 1
                else:
                    # After a track disappears for tracking_thresh frames
                    # Run LOI on no longer tracking IDs
                    
                    self._line_of_interest(results[0].orig_shape[1], cur_clip, track_id, self.track_history[track_id])
                    del self.prev_track_ids[track_id]
                    del self.track_history[track_id]
            
            # Plot the tracks
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = self.track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                
                if track_id not in self.prev_track_ids:
                    self.prev_track_ids[track_id] = self.tracking_thresh
                
                #if len(track) > 30:  # retain 90 tracks for 90 frames
                #    track.pop(0)
    
                # Draw the tracking lines
                #points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                #cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)
            
            #out_vid.write(annotated_frame)

        #out_vid.release()
        return self.salm_count

    def _line_of_interest(self, f_width, cur_clip, track_id, track):
        # Check start and end of track ID
        # Count if start and end are on either sides of the LOI
        half_width = f_width / 2
        first_track_x = track[0][0]
        last_track_x = track[-1][0]
        classes = self.dataloader.classes()
        if first_track_x < half_width and last_track_x >= half_width:
            # Counted going to the right
            # TODO: Figure out classes
            self.salm_count.loc[cur_clip.name, self.RIGHT_PRE + classes[0]] += 1
        elif first_track_x > half_width and last_track_x <= half_width:
            # Counted going to the left
            self.salm_count.loc[cur_clip.name, self.LEFT_PRE + classes[0]] += 1


In [6]:
from pysalmcount.imagedirloader import ImageDirLoader as Iml
from pysalmcount.salmon_counter import SalmonCounter as Sc

with open('./export_combined_bear_kitwanga_all_yolo/combined_bear_kitwanga_all.yaml', 'r') as file:
    data = yaml.safe_load(file)
d = Iml('/mnt/shiorissd4tb/masamim/export_kitwanga_all_yolo/test/09-02-2020_07-49-21_m_salmon_camera/', '.', data['names'])
counter = Sc('train_57_no_filt/weights/best.pt', d)
counter.count()


image 1/1 /mnt/shiorissd4tb/shared/export_kitwanga_all_yolo/test/09-02-2020_07-49-21_m_salmon_camera/obj_train_data/frame_000000.jpg: 384x640 (no detections), 78.2ms
Speed: 4.3ms preprocess, 78.2ms inference, 19.4ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /mnt/shiorissd4tb/shared/export_kitwanga_all_yolo/test/09-02-2020_07-49-21_m_salmon_camera/obj_train_data/frame_000001.jpg: 384x640 (no detections), 5.4ms
Speed: 1.4ms preprocess, 5.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /mnt/shiorissd4tb/shared/export_kitwanga_all_yolo/test/09-02-2020_07-49-21_m_salmon_camera/obj_train_data/frame_000002.jpg: 384x640 (no detections), 5.0ms
Speed: 1.4ms preprocess, 5.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /mnt/shiorissd4tb/shared/export_kitwanga_all_yolo/test/09-02-2020_07-49-21_m_salmon_camera/obj_train_data/frame_000003.jpg: 384x640 (no detections), 5.0ms
Speed: 1.4ms preprocess, 5.0ms inference, 0.4

,l_Coho,r_Coho,l_Bull,r_Bull,l_Rainbow,r_Rainbow,l_Sockeye,r_Sockeye,l_Pink,r_Pink,...,l_Chum,r_Chum,l_Steelhead,r_Steelhead,l_Jack Chinook,r_Jack Chinook,l_Lamprey,r_Lamprey,l_Cutthroat,r_Cutthroat
filename,,,,,,,,,,,,,,,,,,,,,
obj_train_data,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# First convert to H264 by runnning `ffmpeg -i output.mp4 test.mp4`

from IPython.display import Video

Video("test.mp4", embed=True, width=1280)